In [2]:
import re
from evaluate import load
import whisper
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [7]:
def read_txt(path_gt, alternative=False, grammar=False):
    file = open(path_gt, encoding="utf-8")
    lines = file.readlines()

    text = []

    for line in lines:

        if grammar:
            buffer = line.replace("[", "")
            buffer = buffer.replace("]", "")
        else:
            buffer = re.sub("[\[].*?[\]]", "", line)

        new_line = buffer.split()

        for word in new_line:
            if alternative == False:
                if word.startswith("("):
                    new_line.remove(word)
            else:
                if word.startswith("("):
                    tmp = word.replace("(", "")
                    tmp = tmp.replace(")", "")
                    new_line[new_line.index(word) - 1] = tmp
                    index = (new_line.index(word))
                    del new_line[index]

        text.append(" ".join(new_line))
    return text

In [8]:
def get_metrics(path_gt, path_org):
    gt_original = os.path.join(path_gt, "transcript_original.txt")
    gt_deutsch = os.path.join(path_gt, "translation_german.txt")
    path_original = os.path.join(path_org, "transcript_original.txt")
    path_deutsch = os.path.join(path_org, "translation_german.txt")

    original = read_txt(path_original)
    deutsch = read_txt(path_deutsch)
    original_truth = read_txt(gt_original)
    deutsch_truth = read_txt(gt_deutsch)

    original = " ".join(original)
    deutsch = " ".join(deutsch)
    original_truth = " ".join(original_truth)
    deutsch_truth = " ".join(deutsch_truth)

    tmp_original = []
    tmp_deutsch = []
    tmp_org_tr = []
    tmp_dts_tr = []

    tmp_original.append(original)
    tmp_deutsch.append(deutsch)
    tmp_org_tr.append(original_truth)
    tmp_dts_tr.append(deutsch_truth)

    wer = load("wer")

    wer_score_deutsch = wer.compute(predictions=tmp_deutsch, references=tmp_dts_tr)
    wer_score_original = wer.compute(predictions=tmp_original, references=tmp_org_tr)

    print("WER Original: " + str(wer_score_original))
    print("WER Deutsch: " + str(wer_score_deutsch))

In [10]:
path_toAnalyze = ""
path_truth = ""

get_metrics(path_toAnalyze, path_truth)

WER Original: 0.0
WER Deutsch: 0.0
